In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD")
)

In [3]:
#테스트 쿼리 실행 - Neo4j 연결이 제대로 작동하는지 확인하기 위한 간단한 테스트
#Cypher 쿼리를 사용하여 Test 레이블을 가진 노드를 생성하고 반환한다.
cypher_query = """
CREATE (n:Test {name: "Hello AuraDB"})
RETURN n
"""

#이 쿼리가 성공적으로 실해오디면 Neo4J DB연결이 정상적으로 작동하는 것임
graph.query(cypher_query)

[{'n': {'name': 'Hello AuraDB'}}]

In [5]:
def reset_database(graph):
    """데이터베이스 초기화하기"""
    #모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")

    #모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    #모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화되었습니다.")

#데이터베이스 초기화
reset_database(graph)

데이터베이스가 초기화되었습니다.


In [6]:
#Neo4j 데이터베이스에 노드 생성하기 (CREATE 구문 사용)
#동일한 노드가 이미 존재해도 중복생성되므로 주의 해야 함
cypher_query = """
// 주석은 아래와 같이 처리할 수 있음
// 노드 레이블 : Person (사람을 나타내는 레이블)
// 노드 속성 정의:
// - name : "서소망" (이름)
// - age : 10 (나이)
// - email : somang@gmail.com (이메일 주소)
// p는 생성된 노드를 참조하는 변수명입니다.

CREATE (p:Person {name : "서소망", age: 10, email : "somang@gmail.com"}) 
"""

graph.query(cypher_query)

[]

### 노드(Node) 표현

In [9]:
#CREATE 구문을 사용하여 여러 노드를 한번에 생성할 수 있음
cypher_query = """
// 노드 레이블 정의 : 
// - Person : 사람을 나타내는 레이블
// - City : 도시를 타나내는 레이블
//
// Person 노드 속성 정의
// - name : 사람의 이름 (문자열)
// - age : 사람의 나이 (숫자)
//
// City 노드 속성 정의 :
// - name : 도시 이름 (문자열)
// - population : 인구수 (숫자)
//
// 변수 a,b,c는 각각 생성된 노드를 참조하는 변수명 입니다.
CREATE
 (a: Person {name: "김철수", age :25}),
 (b: Person {name: "이영희", age :28}),
 (c: City {name : "서울", population: 9700000})
"""

#graph.query() 를 사용하여 Cypher 쿼리를 실행합니다.
#이 명령은 Neo4j 데이터베이스에 세 개의 노드를 한 번에 생성합니다.
graph.query(cypher_query)

[]

### 관계 (Relationship) 표현

In [11]:
#기존 노드 사이에 관계 생성하기
cypher_query="""
// 관계 레이블 : KNOWS (알고 있다는 관계를 나타냄)
// 관계 속성 : since (관계가 시작된 시점을 타나내는 속성)
// 관계 방향 : a -> b (서소망이 김철수를 알고 있는 방향성)
// 관계 속성 값 : since : 2020 (2020년부터 관계가 시작됨)
// 관계 생성 의미 : 서소망은 김철수를 2020년부터 알게 됨

// MATCH 구문으로 두 개의 기존 노드를 찾음
MATCH
    (a: Person {name: "서소망"}), // Person 레이블을 가진 이름이 서소망인 노드를 찾아 변수 a에 할당
    (b: Person {name: "김철수"}) // Person 레이블을 가진 이름이 김철수인 노드를 찾아 변수 b에 할당

//CREATE 구문으로 두 노드 사이에 방향성 있는 관계 생성
CREATE (a)-[r:KNOWS {since: 2020}] -> (b) // a에서 b로 향하는 KNOWS 타입의 관계 r 생성, since 속성 추가

//RETURN 구문으로 노드와 관계 정보 반환
RETURN a,b,r // 생성된 관계와 연결되 두 정보를 함께 반환
"""

graph.query(cypher_query)

[{'a': {'name': '서소망', 'age': 10, 'email': 'somang@gmail.com'},
  'b': {'name': '김철수', 'age': 25},
  'r': ({'name': '서소망', 'age': 10, 'email': 'somang@gmail.com'},
   'KNOWS',
   {'name': '김철수', 'age': 25})}]

In [ ]:
# 노드와 관계를 한번에 생성하는 Cypher 쿼리
cypher_query = """
// 노드 레이블 : Person, City (각각 사람과 도시를 나타내는 레이블)
// CREATE 구문으로 노드와 관계를 동시에 생성
CREATE
    // a 변수에 Person 레이블을 가진 노드 생성, 속성으로 name = "이삼백" 설정
    (a: Person {name: "이삼백"})-[:LIVES_IN]-> 
    
    // c 변수에 City 레이블을 가진 노드 생성, 속성으로 name ="도쿄" 설정
    // LIVES_IN 관계는 '살고 있다'는 의미로 이삼백이 도쿄에 거주함을 나타냄
    (c: City {name: "도쿄"})
// RETURN 구문으로 생성된 두 노드 정보 반환
RETURN a,c
"""

graph.query(cypher_query)

[{'a': {'name': '이삼백'}, 'c': {'name': '도쿄'}}]